In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import gmaps.datasets

# Import API key
from api_keys import g_key

In [9]:
# Read .csv file of final school data
school_file = os.path.join ("./Resources/school_grades_final.csv")
school_grades = pd.read_csv(school_file)

home_file = os.path.join ("../GB/Resources/All_Hill_Prop_Value.csv")
home_values = pd.read_csv(home_file)

In [2]:
# Configure gmaps
# gmaps.configure(api_key = g_key)
# url = "https://maps.googleapis.com/maps/api/place/findplacefromtext/json?""
# query = url + 

In [10]:
school_grades.head()

,Unnamed: 0,District Number,District Name,School Number,School Name,Total Points Earned,Total Components,Percent of Total Possible Points,Percent Tested,Grade 2019,...,Grade 2017,Grade 2016,Informational Baseline Grade 2015,Charter School,Title I,Alternative/ESE Center School,School Type,Percent of Minority Students,Percent of Economically Disadvantaged Students,ZIP
0,1420,29,HILLSBOROUGH,42,FOREST HILLS ELEMENTARY SCHOOL,338,7,48,100,C,...,D,F,F,NO,YES,N,1,85.6,94.6,33612
1,1422,29,HILLSBOROUGH,51,SHEEHY ELEMENTARY SCHOOL,351,7,50,100,C,...,D,D,F,NO,YES,N,1,97.4,98.0,33610
2,1424,29,HILLSBOROUGH,54,CORR ELEMENTARY SCHOOL,315,7,45,100,C,...,C,C,D,NO,YES,N,1,79.1,78.0,33534
3,1426,29,HILLSBOROUGH,56,DAVIS ELEMENTARY SCHOOL,333,7,48,100,C,...,C,C,C,NO,YES,N,1,85.8,90.5,33615
4,1427,29,HILLSBOROUGH,59,FISHHAWK CREEK ELEMENTARY SCHOOL,497,7,71,100,A,...,A,A,A,NO,NO,N,1,30.6,13.8,33547


In [22]:
# Add column for average property value in school dataframe
school_grades['Avg Prop Value']= ""
school_grades

,Unnamed: 0,District Number,District Name,School Number,School Name,Total Points Earned,Total Components,Percent of Total Possible Points,Percent Tested,Grade 2019,...,Grade 2016,Informational Baseline Grade 2015,Charter School,Title I,Alternative/ESE Center School,School Type,Percent of Minority Students,Percent of Economically Disadvantaged Students,ZIP,Avg Prop Value
0,1420,29,HILLSBOROUGH,42,FOREST HILLS ELEMENTARY SCHOOL,338,7,48,100,C,...,F,F,NO,YES,N,1,85.6,94.6,33612,
1,1422,29,HILLSBOROUGH,51,SHEEHY ELEMENTARY SCHOOL,351,7,50,100,C,...,D,F,NO,YES,N,1,97.4,98.0,33610,
2,1424,29,HILLSBOROUGH,54,CORR ELEMENTARY SCHOOL,315,7,45,100,C,...,C,D,NO,YES,N,1,79.1,78.0,33534,
3,1426,29,HILLSBOROUGH,56,DAVIS ELEMENTARY SCHOOL,333,7,48,100,C,...,C,C,NO,YES,N,1,85.8,90.5,33615,
4,1427,29,HILLSBOROUGH,59,FISHHAWK CREEK ELEMENTARY SCHOOL,497,7,71,100,A,...,A,A,NO,NO,N,1,30.6,13.8,33547,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128,1627,29,HILLSBOROUGH,4801,WILSON ELEMENTARY SCHOOL,284,7,41,99,C,...,C,C,NO,YES,N,1,72.2,90.5,33563,
129,1628,29,HILLSBOROUGH,4841,WIMAUMA ELEMENTARY SCHOOL,332,7,47,99,C,...,C,C,NO,YES,N,1,92.1,95.4,33598,
130,1629,29,HILLSBOROUGH,4921,WITTER ELEMENTARY SCHOOL,259,7,37,99,D,...,C,D,NO,YES,N,1,95.9,96.1,33612,
131,1630,29,HILLSBOROUGH,4941,WOODBRIDGE ELEMENTARY SCHOOL,315,7,45,100,C,...,B,C,NO,YES,N,1,90.1,89.7,33615,


In [11]:
home_values

,Unnamed: 0,SEQ_NO,PARCEL_ID,PropertyValue,NBRHD_CD,CENSUS_BK,PropAddress,City,ZipCode
0,0,1,153323ZZZ000000000200A,4628604,22700200,120579900000,EGMONT KEY,ST PETERSBURG,33715
1,3,4,172701001000000000001U,147699,21100700,120570115091,19507 PINE TREE RD,ODESSA,33556
2,4,5,172701001000000000010U,140829,21100700,120570115092,19931 ANGEL LN,ODESSA,33556
3,5,6,172701001000000000011U,462229,21100700,120570115092,19859 ANGEL LN,ODESSA,33556
4,6,7,172701001000000000012U,496434,21100700,120570115091,7320 W COUNTY LINE RD,ODESSA,33556
...,...,...,...,...,...,...,...,...,...
487178,501068,504373,223233ZZZ000005341400U,2283420,22900400,120570139032,21694 TAYLOR GILL RD,LITHIA,33547
487179,501069,504374,223233ZZZ000005341500U,212500,22900400,120570139032,1,LITHIA,33547
487180,501070,504375,223233ZZZ000005341600U,291900,22900400,120570139032,1,LITHIA,33547
487181,501071,504376,223234ZZZ000005341800U,4261405,42900100,120570139032,11200 S 37 RD,LITHIA,33547


In [24]:
home_average = home_values.groupby(['ZipCode'])
# home_average.mean().head(50)

home_average_df = pd.DataFrame(home_average["PropertyValue"].mean())
home_average_df = home_average_df.reset_index()

home_average_df

,ZipCode,PropertyValue
0,33503,3.817770e+05
1,33510,2.253990e+05
2,33511,3.161239e+05
3,33527,2.573360e+05
4,33534,2.179362e+05
...,...,...
59,33715,4.628604e+06
60,33810,4.756856e+04
61,33849,2.907302e+06
62,33854,1.314630e+05


In [28]:
# Add average property value to school dataframe
for i, row in home_average_df.iterrows():
    school_grades.loc[school_grades['ZIP'] == row['ZipCode'], "Avg Prop Value"] = row['PropertyValue']

school_grades

,Unnamed: 0,District Number,District Name,School Number,School Name,Total Points Earned,Total Components,Percent of Total Possible Points,Percent Tested,Grade 2019,...,Grade 2016,Informational Baseline Grade 2015,Charter School,Title I,Alternative/ESE Center School,School Type,Percent of Minority Students,Percent of Economically Disadvantaged Students,ZIP,Avg Prop Value
0,1420,29,HILLSBOROUGH,42,FOREST HILLS ELEMENTARY SCHOOL,338,7,48,100,C,...,F,F,NO,YES,N,1,85.6,94.6,33612,287204
1,1422,29,HILLSBOROUGH,51,SHEEHY ELEMENTARY SCHOOL,351,7,50,100,C,...,D,F,NO,YES,N,1,97.4,98.0,33610,261973
2,1424,29,HILLSBOROUGH,54,CORR ELEMENTARY SCHOOL,315,7,45,100,C,...,C,D,NO,YES,N,1,79.1,78.0,33534,217936
3,1426,29,HILLSBOROUGH,56,DAVIS ELEMENTARY SCHOOL,333,7,48,100,C,...,C,C,NO,YES,N,1,85.8,90.5,33615,257069
4,1427,29,HILLSBOROUGH,59,FISHHAWK CREEK ELEMENTARY SCHOOL,497,7,71,100,A,...,A,A,NO,NO,N,1,30.6,13.8,33547,321108
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128,1627,29,HILLSBOROUGH,4801,WILSON ELEMENTARY SCHOOL,284,7,41,99,C,...,C,C,NO,YES,N,1,72.2,90.5,33563,220935
129,1628,29,HILLSBOROUGH,4841,WIMAUMA ELEMENTARY SCHOOL,332,7,47,99,C,...,C,C,NO,YES,N,1,92.1,95.4,33598,198097
130,1629,29,HILLSBOROUGH,4921,WITTER ELEMENTARY SCHOOL,259,7,37,99,D,...,C,D,NO,YES,N,1,95.9,96.1,33612,287204
131,1630,29,HILLSBOROUGH,4941,WOODBRIDGE ELEMENTARY SCHOOL,315,7,45,100,C,...,B,C,NO,YES,N,1,90.1,89.7,33615,257069
